In [2]:
%env https_proxy=http://172.16.2.250:3128

env: https_proxy=http://172.16.2.250:3128


In [3]:
from bs4 import BeautifulSoup
import requests
from time import sleep

In [14]:
def scrape_from_page(url, category):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')    
    title = soup.find("header", {"class" : "article-title"}).find("h1").get_text()
    content = " ".join([elt.get_text() for elt in soup.find("div", {"id" : "article_content"}).find_all("p")])
    summary = soup.find("h2", {"class" : "subline"}).get_text()
    date = soup.time.attrs['datetime'][:10]

    return [title, date, url, category, summary, content]

In [5]:
def get_topic_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    urls = list(set([elt["href"] for elt in soup.find("section", {"id" : "repo-list"}).find_all("a")]))
    return urls

In [15]:
def scrape(page_no, topic):
    URL = f"https://www.ibtimes.co.in/{topic}/page/{page_no}"
    url_list = get_topic_pages(requests.get(URL))
    sleep(1)

    data = []

    for url in url_list:
        dt = scrape_from_page(url, topic)
        if (dt):
            data.append(dt)
            sleep(1)

    return data

In [16]:
def scrape_loop(start_pg, end_pg, topic):
    data = []
    for pg in range(start_pg, end_pg):
        data += scrape(pg, topic)
        
    return data

In [17]:
dt = scrape_loop(2, 3, "science")

In [32]:
# 120 articles in 3 mins. - 10 pages
#  40 articles in 1 min.